<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. Launch the web UI with Google Drive Storage (CPU/GPU Compatible)

#@markdown If unsure about the branch, write "main" or leave it blank.

import os
import re
import subprocess
from pathlib import Path
from google.colab import drive
import shutil

# Mount Google Drive
print("\033[1;32;1m\n --> Mounting Google Drive...\033[0;37;0m\n")
drive.mount('/content/drive')

# Set up directories in Google Drive
DRIVE_PATH = '/content/drive/MyDrive/text-generation-webui'
os.makedirs(DRIVE_PATH, exist_ok=True)

os.environ.pop('PYTHONPATH', None)
os.environ.pop('MPLBACKEND', None)

# Check if installation exists in Drive, if not install there
if not Path(f'{DRIVE_PATH}/.git').exists():
  print("\033[1;32;1m\n --> Installing the web UI in Google Drive. This will take a while, but only needs to be done once.\033[0;37;0m\n")

  %cd /content/drive/MyDrive
  !git clone https://github.com/gitleon8301/MY-AI-Gizmo-working/tree/main
  %cd text-generation-webui

  # Install the project in an isolated environment
  !GPU_CHOICE=A \
  LAUNCH_AFTER_INSTALL=FALSE \
  INSTALL_EXTENSIONS=FALSE \
  ./start_linux.sh
else:
  print("\033[1;32;1m\n --> Found existing installation in Google Drive. Skipping installation.\033[0;37;0m\n")
  %cd {DRIVE_PATH}

# Detect GPU availability
try:
    import torch
    has_gpu = torch.cuda.is_available()
    if has_gpu:
        gpu_name = torch.cuda.get_device_name(0)
        print(f"\033[1;32;1m ✓ GPU detected: {gpu_name}\033[0;37;0m")
    else:
        print("\033[1;33;1m ⚠ No GPU detected - using CPU mode\033[0;37;0m")
except:
    has_gpu = False
    print("\033[1;33;1m ⚠ No GPU detected - using CPU mode\033[0;37;0m")

# Parameters - adjust based on your needs
#@markdown ### Model Selection (choose CPU or GPU models)
#@markdown **For GPU:** EXL2, GPTQ, AWQ formats
#@markdown **For CPU:** GGUF formats

model_url = "https://huggingface.co/bartowski/gemma-2-9b-it-GGUF" #@param {type:"string"}
branch = "main" #@param {type:"string"}

# Auto-configure flags based on GPU availability
if has_gpu:
    command_line_flags = "--n-gpu-layers 128 --api" #@param {type:"string"}
else:
    command_line_flags = "--cpu --api" #@param {type:"string"}

# Always enable API and sharing
if '--api' not in command_line_flags:
    command_line_flags += ' --api'
if '--share' not in command_line_flags:
    command_line_flags += ' --share'

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # Parse model info
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')

    if branch.strip() not in ['', 'main']:
        output_folder += f"_{branch}"

    model_path = Path(f"{DRIVE_PATH}/models/{output_folder}")

    # Check if model exists and is complete
    if model_path.exists():
        # Check if there are any actual model files
        model_files = list(model_path.glob('*.safetensors')) + \
                     list(model_path.glob('*.bin')) + \
                     list(model_path.glob('*.gguf'))

        if model_files:
            print(f"\033[1;32;1m ✓ Model already exists in Drive: {output_folder}\033[0;37;0m\n")
        else:
            print(f"\033[1;33;1m ⚠ Model folder exists but appears incomplete. Cleaning and re-downloading...\033[0;37;0m\n")
            shutil.rmtree(model_path)
            print(f"\033[1;32;1m --> Downloading model to Drive: {output_folder}\033[0;37;0m\n")
            if branch.strip() not in ['', 'main']:
                !python download-model.py {model_url} --branch {branch} --clean
            else:
                !python download-model.py {model_url} --clean
    else:
        print(f"\033[1;32;1m --> Downloading model to Drive: {output_folder}\033[0;37;0m\n")
        if branch.strip() not in ['', 'main']:
            !python download-model.py {model_url} --branch {branch}
        else:
            !python download-model.py {model_url}
else:
    output_folder = ""

# Start the web UI and capture URLs
cmd = f"./start_linux.sh {command_line_flags}"
if output_folder != "":
    cmd += f" --model {output_folder}"

print("\n" + "\033[1;35;1m" + "="*80)
print("STARTING WEB UI - WATCH FOR URLS BELOW")
print("="*80 + "\033[0;37;0m\n")

# Run the command
process = subprocess.Popen(
    cmd,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Monitor output for URLs
local_url = None
public_url = None
urls_displayed = False

for line in iter(process.stdout.readline, ''):
    print(line, end='')

    # Capture local URL patterns
    if not local_url:
        local_match = re.search(r'(http://(?:127\.0\.0\.1|0\.0\.0\.0|localhost):\d+)', line)
        if local_match:
            local_url = local_match.group(1)

    # Capture public URL patterns
    if not public_url:
        public_match = re.search(r'(https://[a-z0-9\-]+\.gradio\.live)', line)
        if public_match:
            public_url = public_match.group(1)

    # Display URLs prominently when both are found
    if local_url and public_url and not urls_displayed:
        print("\n" + "\033[1;32;1m" + "="*80)
        print("🎉 WEB UI IS READY!")
        print("="*80)
        print(f"\n📍 LOCAL URL (Colab):  {local_url}")
        print(f"🌐 PUBLIC URL (Share): {public_url}")
        print(f"\n💡 Click the PUBLIC URL to access from anywhere!")
        print("="*80 + "\033[0;37;0m\n")
        urls_displayed = True

process.wait()